In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

In [ ]:
file_path = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss63_09Bparams_2023-06-22_uniform-09b_n500_r100_f40_l10'
hdf = h5py.File(file_path+'/sam_lib.hdf5', 'r')

print(hdf.keys())
sample_params = hdf['sample_params'][...]
hc_ss = hdf['hc_ss'][...]
hc_bg = hdf['hc_bg'][...]
sspar = hdf['sspar'][...]
bgpar = hdf['bgpar'][...]
fobs_gw_cents = hdf['fobs'][:]
hdf.close()

dur = 1/fobs_gw_cents[0]
fobs_gw_cents, fobs_gw_edges = holo.utils.pta_freqs(dur, num=len(fobs_gw_cents))
nsamps, nfreqs, nreals, nloudest = [*hc_ss.shape]


In [ ]:
npz = np.load(file_path+'/PS_Uniform_09B.pspace.npz', allow_pickle=True)
print(npz.files)
param_names = npz['param_names']
print(param_names)
print(npz['class_name'])
lib_name = '%s v%s, %d samples' % (npz['class_name'], npz['librarian_version'], nsamps)
print(lib_name)
# print(npz['sam_shape'])
npz.close()

# Load detstats

In [ ]:
npz = np.load('/Users/emigardiner/GWs/holodeck/output/brc_output/ss63_09Bparams_2023-06-22_uniform-09b_n500_r100_f40_l10/detstats/clbrt_psrs40/detstats_lib.npz')
print(npz.files)
# print(npz['snr_ss'].shape)
dp_ss = npz['dp_ss']
dp_bg = npz['dp_bg']
df_ss = npz['df_ss']
df_bg = npz['df_bg']
ev_ss = npz['ev_ss']
nskies = dp_ss.shape[-1]
print(f"{ev_ss.shape=}, {dp_ss.shape=}")
npz.close()

## Make plot shapes (N,R,S)

In [ ]:
ratio = dp_ss/dp_bg[:,:,np.newaxis]
print(f"{ratio.shape=}")

In [ ]:
thard = np.repeat(sample_params[:,0], nreals*nskies).reshape(nsamps, nreals, nskies)

In [ ]:
fig, ax = plot.figax(xlabel='hard_time', ylabel='ev_ss/dp_bg')
ax.scatter(thard, ratio, s=1)

In [ ]:
mchar = sample_params[:,2]

In [ ]:
def normalize(xx, log=True, diverging=False, center=None):
    xmin = np.min(xx)
    xmax = np.max(xx)
    if log:
        xmin = np.log10(xmin)
        xmax = np.log10(xmax)
    print(f"{xmin=}, {xmax=}")
    if diverging:
        norm = mpl.colors.TwoSlopeNorm(vmin=xmin, vmax=xmax, center=center)
    else:
        norm = mpl.colors.Normalize(vmin=xmin, vmax=xmax)
    return norm


In [ ]:

mchar = np.repeat(sample_params[:,2], nreals*nskies)
thard = np.repeat(sample_params[:,0], nreals*nskies)

In [ ]:
cmap='PiYG'
def scatter_with_color(xx, yy, cc, norm, xlabel=None, ylabel=None, clabel=None,
                       xscale='log', yscale='log', ccent=0, cmap='PiYG'):
    fig, ax = plot.figax(xlabel=xlabel, ylabel=ylabel, yscale=yscale, xscale=xscale)
    sax = ax.scatter(xx.flatten(), yy.flatten(), c=cc.flatten(), cmap=cmap, norm=norm)
    cbar = plt.colorbar(sax, ax=ax, label=clabel)
    return fig
# create the colorbar
# # Create the colorbar
# smap = plt.cm.ScalarMappable(cmap='PiYG', norm=norm)
# cbar = fig.colorbar(smap, ax=ax, fraction=0.1, shrink = 0.8)

In [ ]:
xx = thard
yy = mchar
cc = np.log10(ratio.flatten())
xlabel = 'hard_time'
ylabel = 'GSMF mchar0 log10'
clabel = 'SS/BG Detprob'
yscale='linear'
xscale='log'
norm = normalize(cc, center=0)
print(f"{xx.shape=}, {yy.shape=}, {cc.shape=}")

fig = scatter_with_color(
    xx, yy, cc, norm, xlabel, ylabel, clabel, 
    xscale=xscale, yscale=yscale, ccent=0, cmap='PiYG')

In [ ]:
xx = thard
yy = np.repeat(sample_params[:,1], nreals*nskies)
cc = np.log10(ratio.flatten())
xlabel = 'hard_time'
ylabel = 'GSMF Phi0'
clabel = 'SS/BG Detprob'
yscale='linear'
xscale='log'
norm = normalize(cc, center=0)
print(f"{xx.shape=}, {yy.shape=}, {cc.shape=}")

fig = scatter_with_color(
    xx, yy, cc, norm, xlabel, ylabel, clabel, 
    xscale=xscale, yscale=yscale, ccent=0, cmap='PiYG')